## Gradient Boosting Model on Weather Station Data

- Gradient builds an additive model in a forward stage-wise fashion, which allows for the optimization of arbitrary differentiable loss functions. 


In [ ]:
# import dependencies
import pandas as pd
import sqlalchemy as sq
import sys, os
import pickle
from imblearn.combine import SMOTEENN
from xgboost import XGBClassifier
from sklearn.ensemble import (  # type: ignore
    GradientBoostingClassifier,
)
from imblearn.ensemble import (  # type: ignore
    RUSBoostClassifier,
)

from sklearn.metrics import (  # type: ignore
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

sys.path.append("../../")
os.chdir("../../")
from ModelBuilderMethods import getConn, extractYears, scaleColumns, encodeColumns
from Models.models_ensemble import getBalancedClassifier, getClassifier

In [ ]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

### <u>**Step 1**</u>: Data Selection

In this step, we would choose the particular data/table, pick attributes from existing tables. Further aggregation/feature engineer can be done here to support the point of the research.

Particular, for this notebook, we grab the following data and merge them (on year, district) into a single table:
- Monthly weather station
- ergot data (downgrade)

In [ ]:
# Set the query text
weatherStationQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_station
"""
)

ergotTargetQuery = sq.text(
    """
    SELECT year, district, downgrade from ergot_sample_feat_eng
"""
)

In [ ]:
conn = getConn("./.env")

stationDf = pd.read_sql(weatherStationQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [ ]:
# merge on year and district
# tempdf = pd.merge(satelliteDf, ergotPrevDf, on=["year", "district"], how="left")
# del satelliteDf
# del ergotPrevDf
# tempdf = satelliteDf
tempdf = stationDf

# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

Then, the district number would be categorized using [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) techniques.

One-hot encoding techniques would take input, which is a categorical value. Then, the features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme. This creates a binary column for each category and returns a sparse matrix or dense array (depending on the sparse_output parameter).


In [ ]:
# encode district
datasetDf["district"] = datasetDf["district"].astype("category")

temp = pd.get_dummies(datasetDf["district"], prefix="district", drop_first=True)
datasetDf = pd.concat([datasetDf, temp], axis=1)

datasetDf = datasetDf.drop(columns=["district"])

del temp

### <u>**Step 2**</u>: Splitting dataset

- We split the whole dataset into the train/test split. Particularly, split them by year (1995 - 2015 for training, 2016 - 2020 for testing) since this is a time series data.

In [ ]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

In [ ]:
# drop year
trainDf = trainDf.drop(columns=["year"])
testDf = testDf.drop(columns=["year"])

### <u>**Step 3**</u>: [Balancing the dataset](https://imbalanced-learn.org/stable/)

- Our dataset is unbalanced and can lead to bias when training/testing. Balacing step would help to eliminate the bias of the dataset, thus provide more reliable results.

In [ ]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

downgrade
False    122202
True       2082
Name: count, dtype: int64
downgrade
False    26307
True      1016
Name: count, dtype: int64


In [ ]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
trainDf = trainDf.fillna(0)

downgrade                    0
1:min_temp_x              1246
1:max_temp_x              1246
1:mean_temp_x             1246
1:min_dew_point_temp      1246
1:max_dew_point_temp      1246
1:mean_dew_point_temp     1246
1:min_humidex             1246
1:max_humidex             1246
1:mean_humidex            1246
1:min_precip              1246
1:max_precip              1246
1:mean_precip             1246
1:min_rel_humid           1246
1:max_rel_humid           1246
1:mean_rel_humid          1246
1:min_stn_press           1246
1:max_stn_press           1246
1:mean_stn_press          1246
1:min_visibility          1246
1:max_visibility          1246
1:mean_visibility         1246
1:max_temp_y              1246
1:min_temp_y              1246
1:mean_temp_y             1246
1:min_total_rain          1246
1:max_total_rain          1246
1:mean_total_rain         1246
1:min_total_snow          1246
1:max_total_snow          1246
1:mean_total_snow         1246
1:min_total_precip        1246
1:max_to

In [ ]:
balancer = SMOTEENN(sampling_strategy=1, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(
    trainDf.drop(columns="downgrade"), trainDf["downgrade"]
)

In [ ]:
# post balancing check
# print value counts downgrade
print(balancedTrainDfY.value_counts())

downgrade
False    115179
True      23757
Name: count, dtype: int64


### <u>**Step 4**</u>: Regularization / Normalization
some blurb about scalers  

1. [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)             
2. [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)  
3. [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)  
4. [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)  
5. [Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)  
6. [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)  
7. [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)  

In [ ]:
# df = pd.DataFrame()
# scaled = scaleColumns(df, ['max_temp'], None, 1)

In [ ]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

### <u>**Step 5**</u>: Gradient Boosting Classifier Model

In [ ]:
ESTIMATORS = 400
DEPTH = 40
CORES = -1
MINSPLSPLIT = 8
MINSAMPLELEAF = 4

gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
balanced_gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
balanced_rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)
balanced_xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)

In [ ]:
gradient_boosting_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_gradient_boosting_model.fit(balancedTrainDfX, balancedTrainDfY)

      Iter       Train Loss   Remaining Time 
         1           0.1563           76.55m
         2           0.1517           76.15m
         3           0.1484           76.18m
         4           0.1459           76.06m
         5           0.1438           75.35m
         6           0.1422           74.86m
         7           0.1408           74.84m
         8           0.1397           73.82m
         9           0.1387           72.84m
        10           0.1378           72.05m
        20           0.1336           66.49m
        30           0.1324           62.26m
        40           0.1320           57.46m
        50           0.1318           77.18m
        60           0.1318           77.62m
        70           0.1318           75.61m
        80           0.1317           72.45m
        90           0.1317           69.56m
       100           0.1317           66.95m
       200           0.1317           35.53m
      Iter       Train Loss   Remaining Time 
        

GradientBoostingClassifier(max_depth=40, n_estimators=400, n_iter_no_change=200,
                           random_state=42, verbose=1)

eval procedure

In [ ]:
rusboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_rusboost_model.fit(balancedTrainDfX, balancedTrainDfY)

RUSBoostClassifier(n_estimators=400, random_state=42, sampling_strategy=0.5)

In [ ]:
xgboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_xgboost_model.fit(balancedTrainDfX, balancedTrainDfY)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=40, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [ ]:
# set nan to 0
testDf = testDf.fillna(0)

In [ ]:
# get predictions

predictions_gradient_boosting = gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_balanced_gradient_boosting = balanced_gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_rusboost = rusboost_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_rusboost = balanced_rusboost_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_xgboost = xgboost_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_xgboost = balanced_xgboost_model.predict(
    testDf.drop(columns="downgrade")
)

In [ ]:
print(pd.DataFrame(predictions_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_balanced_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_rusboost).value_counts())
print(pd.DataFrame(predictions_balanced_rusboost).value_counts())
print(pd.DataFrame(predictions_xgboost).value_counts())
print(pd.DataFrame(predictions_balanced_xgboost).value_counts())

False    27323
Name: count, dtype: int64
False    16911
True     10412
Name: count, dtype: int64
False    23040
True      4283
Name: count, dtype: int64
False    24714
True      2609
Name: count, dtype: int64
0    27323
Name: count, dtype: int64
0    26222
1     1101
Name: count, dtype: int64


In [ ]:
# get accuracy precision recall f1 roc_auc
printMetrics(
    "sk GB imbalanced train set", testDf["downgrade"], predictions_gradient_boosting
)
printMetrics(
    "imb GB balanced train set",
    testDf["downgrade"],
    predictions_balanced_gradient_boosting,
)
printMetrics("sk RUS imbalanced train set", testDf["downgrade"], predictions_rusboost)
printMetrics(
    "imb RUS balanced train set", testDf["downgrade"], predictions_balanced_rusboost
)
printMetrics("sk XGB imbalanced train set", testDf["downgrade"], predictions_xgboost)
printMetrics(
    "imb XGB balanced train set", testDf["downgrade"], predictions_balanced_xgboost
)

sk GB imbalanced train set
Accuracy:  0.9628152106284082
Precision:  0.0
Recall:  0.0
F1:  0.0
ROC AUC:  0.5
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      1.00      0.98     26307
        True       0.00      0.00      0.00      1016

    accuracy                           0.96     27323
   macro avg       0.48      0.50      0.49     27323
weighted avg       0.93      0.96      0.94     27323


imb GB balanced train set
Accuracy:  0.6084617355341654
Precision:  0.03505570495582021
Recall:  0.359251968503937
F1:  0.06387819390969549
ROC AUC:  0.488669204687594
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.62      0.75     26307
        True       0.04      0.36      0.06      1016

    accuracy                           0.61     27323
   macro avg       0.50      0.49      0.41     27323
weighted avg       0.93      0.61      0.73     27323


sk RUS imbalanced

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: Undefine

Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.90      0.93     26307
        True       0.03      0.08      0.04      1016

    accuracy                           0.87     27323
   macro avg       0.50      0.49      0.49     27323
weighted avg       0.93      0.87      0.90     27323


sk XGB imbalanced train set
Accuracy:  0.9628152106284082
Precision:  0.0
Recall:  0.0
F1:  0.0
ROC AUC:  0.5
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      1.00      0.98     26307
        True       0.00      0.00      0.00      1016

    accuracy                           0.96     27323
   macro avg       0.48      0.50      0.49     27323
weighted avg       0.93      0.96      0.94     27323


imb XGB balanced train set
Accuracy:  0.9258134172675035
Precision:  0.04087193460490463
Recall:  0.04429133858267716
F1:  0.042512990080302314
ROC AUC:  0.5020749656763311
Classificat

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


print model performance metrics on test data